In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from models.managers.db_manager import connect_to_bdd
from src.data.utils.constants import id_to_party_dict

conn = connect_to_bdd()

# Requête pour election_2022_t1 et sauvegarde dans df_election
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Fermeture de la connexion
conn.close()

# Mapper les noms des partis politiques aux identifiants numériques
party_to_id_dict = {v: k for k, v in id_to_party_dict.items()}
df_election["winner"] = df_election["winner"].map(party_to_id_dict)

# Convertir la colonne "winner" en float
df_election["winner"] = df_election["winner"].astype(float)

label_encoder = LabelEncoder()

# Exclure les données du département de l'Yonne (89) du jeu d'entraînement
df_train = df_election[df_election["code_postal"] != 89]

# Sélectionner uniquement les données du département de l'Yonne (89) pour les tests
df_test_yonne = df_election[df_election["code_postal"] == 89]

# Ajout de 10 autres départements au département de l'Yonne (89) pour les tests
other_departments = [90, 91, 92, 93, 94, 95, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
for department in other_departments:
    df_test_yonne = pd.concat([df_test_yonne, df_election[df_election["code_postal"] == department]])

# Exclure les données des départements ajoutés des données d'entraînement
df_train = df_train[~df_train["code_postal"].isin(other_departments)]

# Séparation des features et de la target pour l'ensemble d'entraînement
X_train = df_train.drop(columns=["Libellé de la commune", "winner"])
y_train = df_train["winner"]

# Séparation des features et de la target pour l'ensemble de test de l'Yonne
X_test_yonne = df_test_yonne.drop(columns=["Libellé de la commune", "winner"])
y_test_yonne = df_test_yonne["winner"]

# Entraînement du modèle avec 30000 itérations
model = LogisticRegression(max_iter=30000)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble d'entraînement
y_pred_train = model.predict(X_train)

# Calcul de l'accuracy du modèle
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Précision du modèle sur l'ensemble d'entraînement:", accuracy_train)

# Prédiction sur les données de test de l'Yonne
y_pred_yonne = model.predict(X_test_yonne)

# Calcul de l'accuracy du modèle sur les données de l'Yonne
accuracy_yonne = accuracy_score(y_test_yonne, y_pred_yonne)
print("Précision du modèle sur les données de l'Yonne:", accuracy_yonne)

/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_2966/4013575320.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)


Précision du modèle sur l'ensemble d'entraînement: 0.5706934114642025
Précision du modèle sur les données de l'Yonne: 0.5981589958158996
